# Libraries & Constants

In [109]:
__author__ = "Aiello Davide"
import random
import logging
from collections import namedtuple
from copy import deepcopy
from itertools import accumulate
from operator import xor
from typing import Callable

# Nim Class

In [104]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [105]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


# Task 1: Fixed rules

In [106]:
def pure_random(state: Nim) -> Nimply: # take a random row and select random elements (always < k)
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    if(state.rows[row] > state.k):
        num_objects = random.randint(1, state.k)
    else:
        num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def shortest_row(state: Nim) -> Nimply:
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > k):
       num_objects = random.randint(1, k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)

def my_strategy(state: Nim) -> Nimply:
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    if any([True for i in possible_moves if i[1] > 1]):
        obj = 0
        while(obj == 0):
            row_num = random.randint(0, len(state.rows) - 1)
            if state.rows[row_num] > 0:
                obj = max([i[1] for i in possible_moves if i[0] == row_num], key=lambda i:i)
                if obj > k:
                    obj = k
                ply = Nimply(row_num, obj)
    else: 
        ply = None
        while ply == None or ply[1] > k:
            ply = Nimply(*possible_moves[random.randint(0, len(possible_moves) - 1)])
    return ply

def shortest_row(state: Nim) -> Nimply: # take the shortest row and select random elements if the matches > k otherwise close the row
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > state.k):
       num_objects = random.randint(1, state.k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)


# Nim-sum

In [107]:
# algorithm taken from professor's code
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)       # numero di righe maggiori di zero     
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]


# Main

In [108]:
logging.getLogger().setLevel(logging.DEBUG)

def evaluate(pure_random: Callable, my_strategy: Callable, NUM_MATCHES : int) -> list:
    w = [0, 0]
    turn = [0, 1]
    for t in turn:
        if t == 0:
            opponent = (my_strategy, pure_random)
        else:
            opponent = (pure_random, my_strategy)
        for m in range(NUM_MATCHES):
            nim = Nim(NIM_SIZE, k)
            player = 0
            while nim:
                ply = opponent[player](nim)
                nim.nimming(ply)
                player = 1 - player
            winner = 1 - player
            if winner == t:
                w[t] += 1
    w[0] = w[0] / NUM_MATCHES
    w[1] = w[1] / NUM_MATCHES
    return w

for n_m in [10, 100, 1000]:
    NIM_SIZE = random.randint(5, 13)
    k = random.randint(2, (NIM_SIZE - 1) * 2 +1)
    w = evaluate(pure_random, my_strategy, n_m)
    logging.info(f"NUM_MATCHES : {n_m} - ratio as player1: {w[0]*100}, ratio as player2: {w[1]*100} with k : {k} (NIM_SIZE = {NIM_SIZE})")

INFO:root:NUM_MATCHES : 10 - ratio as player1: 50.0, ratio as player2: 80.0 with k : 5 (NIM_SIZE = 11)
INFO:root:NUM_MATCHES : 100 - ratio as player1: 70.0, ratio as player2: 65.0 with k : 12 (NIM_SIZE = 12)
INFO:root:NUM_MATCHES : 1000 - ratio as player1: 66.7, ratio as player2: 68.8 with k : 10 (NIM_SIZE = 8)
